# Trabajo Final - Planning - MAIE
En la presente notebook se realizará el trabajo final de la materia Planning de la MAIE, y se explicará paso a paso su resolución. Primero, seleccionar la fecha a procesar en la variable ***fecha*** de la forma YYYY-MM-DD en formato string

In [19]:
# ----- Elegir fecha a procesar -----
fecha = '2023-01-06'

# ----------------------------------------------------------
# ----- NO MODIFICAR: Importación de librarías -------------
# ----------------------------------------------------------
import os
import sys
import glob
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
gdal.UseExceptions()

# -----------------------------------------
# ----- NO MODIFICAR: CHEQUEO -------------
# -----------------------------------------
path = os.getcwd()

# Verifico que exista la carpeta de datos, si no lanzo error
if not os.path.exists(f'{path}/datos'):
    raise Exception(
        '''
        No existe la carpeta "datos" en el directorio actual.
        Verificar que el path de ejecución sea donde se clonó el repositorio
        ''')
path_datos = f'{path}/datos'

# Verifico que exista la carpeta de procesamiento, si no la creo
if not os.path.exists(f'{path}/procesamiento'):
    os.system(f'mkdir {path}/procesamiento')
path_proc = f'{path}/procesamiento'

# Verifico que exista la carpeta de resultados, si no la creo
if not os.path.exists(f'{path}/plots'):
    os.system(f'mkdir {path}/plots')
path_plots = f'{path}/plots'

# Verifico que exista la fecha a procesar
# Los-Molinos_2023-01-06_0_all_bands
datos_path_list = glob.glob(f'{path_datos}/Los-Molinos_{fecha}_*.zip')
if len(datos_path_list) == 0:
    raise Exception(f'No se encontraron datos para la fecha {fecha}')
elif len(datos_path_list) == 1:
    path_file = datos_path_list[0]
    print(f'Existe un único archivo de datos para la fecha {fecha}. Continuar ejecución')
else:
    raise ValueError(f'Existen {len(datos_path_list)} archivos para la fecha {fecha}. Verificar, se aborta la ejecución')

Existe un único archivo de datos para la fecha 2023-01-06. Continuar ejecución


A continuación, descomprimirá el zip en la carpeta ***procesamiento*** y obtengo los links de cada banda

In [31]:
# Creo subcarpeta de extacción, si existe, elimino el contenido
path_proc_fecha = f'{path_proc}/{fecha}'
if not os.path.exists(path_proc_fecha):
    os.system(f'mkdir {path_proc_fecha}')
else:
    os.system(f'rm -r {path_proc_fecha}/*')

# Decomprimo path_file a path_proc    
with zipfile.ZipFile(path_file, 'r') as zip_ref:
    zip_ref.extractall(path_proc_fecha)
    print('-'*80)
    print(f'Archivo:\n{path_file.replace(path, '')}\nDescomprimido en:\n{path_proc_fecha.replace(path, '')}/')
    print('-'*80)

# Obtengo los paths de las bandas
path_bands = glob.glob(f'{path_proc_fecha}/*.tif')

# Controlo que se hayan encontrado archivos
if len(path_bands) == 0:
    raise ValueError('No se encontraron los archivos decomprimidos, revisar código')

print('Las bandas encontradas son:')
for band in path_bands:
    print(band.split('/')[-1].split('.')[1])

--------------------------------------------------------------------------------
Archivo:
/datos/Los-Molinos_2023-01-06_0_all_bands.zip
Descomprimido en:
/procesamiento/2023-01-06/
--------------------------------------------------------------------------------
Las bandas encontradas son:
SR_B1
SR_B2
SR_B3
SR_B4
SR_B5
SR_B6
SR_B7
ST_B10
QA_PIXEL


Genero índice de MNDWI para generar la máscara de agua/no-agua con su fórmula para Landsat 8:

$MNDWI = \frac{Banda 3 – Banda 6}{Banda 3 + Banda 6}$

Genero índice de NDVI con el recorte realizado con su fórmula para Lansat 8:

$NDVI = \frac{Banda 5 – Banda 4}{Banda 5 + Banda 4}$